# Previsão de palavras utilizando NLP.
<br><br>
O objetivo deste notebook é utilizar processamento de lingugagem natural para fazer predição de palavras utilizando como base de aprendizado 5 textos das ciências humanas voltados para área da antropologia e que falam de povos "primitivos" como tratava a ciência na época. No final faremos algumas predições em cima de palavras chaves e alguma análise do processo.
<br>
Os livros utilizados nos treinamentos estarão descritos no decorrer do código.
<br>
Este código é uma adaptação do código disponível em: <a> https://github.com/jackfrost1411/next_word_suggestor <a>


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re # para mexer com regexp
from keras.utils.all_utils import to_categorical
from urllib.request import urlopen
from keras import models


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Textos do projeto gutenberg:

text_1 = urlopen('https://www.gutenberg.org/ebooks/6693.txt.utf-8') #People of Africa
text_2 = urlopen('https://www.gutenberg.org/files/64088/64088-0.txt') #Children of Africa
text_3 = urlopen('https://www.gutenberg.org/files/55974/55974-0.txt') #Anthropophagy
text_4 = urlopen('https://www.gutenberg.org/ebooks/58475.txt.utf-8') #Sexual Life of Primitive People
text_5 = urlopen('https://www.gutenberg.org/files/59922/59922-0.txt') #Stories of the Cave People



In [ ]:
texto_1 = (text_1.read()).decode('utf-8')
texto_2 = (text_2.read()).decode('utf-8')
texto_3 = (text_3.read()).decode('utf-8')
texto_4 = (text_4.read()).decode('utf-8')
texto_5 = (text_5.read()).decode('utf-8')





In [ ]:
#Colocando todos os textos na mesma variável:

data = texto_1 +texto_2 + texto_3 + texto_4 + texto_5
del texto_1, texto_2, texto_3, texto_4, texto_5

In [ ]:

print(data)

﻿The Project Gutenberg EBook of People of Africa, by Edith A. How

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**eBooks Readable By Both Humans and By Computers, Since 1971**

*****These eBooks Were Prepared By Thousands of Volunteers!*****


Title: People of Africa

Author: Edith A. How


In [ ]:
#Aplicando o mesmo algoritmo de limpeza que o notebook original:

cleaned = re.sub(r'\W+', ' ', data).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

print(vocabulary_size)

10347


In [ ]:
# Verificando algumas sequências:

k = np.random.choice(range(len(n_sequences)))
for i in sequences[k]:
  print(tokens[i])

to
ebook
16
artwork


In [ ]:
#Prepara os inputs para entrar na rede:

train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size) # one hot encoding
seq_len = train_inputs.shape[1]

print(train_inputs.shape)

(132145, 3)


Iremos treinar 3 redes diferentes, uma com tamanho padrão, uma compacta e uma mais robusta. No final compararemos os resultados de predições com as 3 versões da rede.

In [ ]:
#Rede de tamanho padrão:

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras import callbacks

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              31041     
                                                                 
 lstm (LSTM)                 (None, 3, 50)             10800     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 10347)             527697    
                                                                 
Total params: 592,288
Trainable params: 592,288
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Treinando:

early = callbacks.EarlyStopping(monitor = 'accuracy',patience= 5)
check = callbacks.ModelCheckpoint(filepath = 'nlp_anthro_v1.h5', monitor = 'accuracy', save_best_only = True, verbose = 1, mode='auto')            



model.fit(train_inputs,train_targets,epochs=500,validation_split=0.3,verbose=1, callbacks= [early,check])

Epoch 1/500
2891/2891 [==============================] - ETA: 0s - loss: 6.8923 - accuracy: 0.0773
Epoch 00001: accuracy improved from -inf to 0.07732, saving model to nlp_anthro_v1.h5
2891/2891 [==============================] - 70s 23ms/step - loss: 6.8923 - accuracy: 0.0773 - val_loss: 6.9896 - val_accuracy: 0.0860
Epoch 2/500
2889/2891 [============================>.] - ETA: 0s - loss: 6.5129 - accuracy: 0.0867
Epoch 00002: accuracy improved from 0.07732 to 0.08670, saving model to nlp_anthro_v1.h5
2891/2891 [==============================] - 64s 22ms/step - loss: 6.5129 - accuracy: 0.0867 - val_loss: 6.9934 - val_accuracy: 0.1061
Epoch 3/500
2891/2891 [==============================] - ETA: 0s - loss: 6.2852 - accuracy: 0.0983
Epoch 00003: accuracy improved from 0.08670 to 0.09827, saving model to nlp_anthro_v1.h5
2891/2891 [==============================] - 65s 23ms/step - loss: 6.2852 - accuracy: 0.0983 - val_loss: 6.9788 - val_accuracy: 0.1116
Epoch 4/500
2891/2891 [===========

In [ ]:
#Treinando o modelo compacto: Menos da metade da quantidade de neorônios da outra rede:

model_1 = Sequential()
model_1.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model_1.add(LSTM(10,return_sequences=True))
model_1.add(LSTM(10))
model_1.add(Dense(10,activation='relu'))
model_1.add(Dense(vocabulary_size, activation='softmax'))
print(model_1.summary())
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 3)              31041     
                                                                 
 lstm_2 (LSTM)               (None, 3, 10)             560       
                                                                 
 lstm_3 (LSTM)               (None, 10)                840       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10347)             113817    
                                                                 
Total params: 146,368
Trainable params: 146,368
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Treinando:

early = callbacks.EarlyStopping(monitor = 'accuracy',patience= 5)
check = callbacks.ModelCheckpoint(filepath = 'nlp_anthro_v2.h5', monitor = 'accuracy', save_best_only = True, verbose = 1, mode='auto')            



model_1.fit(train_inputs,train_targets,epochs=500,validation_split=0.3,verbose=1, callbacks= [early,check])

Epoch 1/500
2889/2891 [============================>.] - ETA: 0s - loss: 6.7530 - accuracy: 0.0774
Epoch 00001: accuracy improved from -inf to 0.07739, saving model to nlp_anthro_v2.h5
2891/2891 [==============================] - 50s 17ms/step - loss: 6.7529 - accuracy: 0.0774 - val_loss: 7.0818 - val_accuracy: 0.0860
Epoch 2/500
2889/2891 [============================>.] - ETA: 0s - loss: 6.5545 - accuracy: 0.0774
Epoch 00002: accuracy did not improve from 0.07739
2891/2891 [==============================] - 47s 16ms/step - loss: 6.5546 - accuracy: 0.0774 - val_loss: 7.1646 - val_accuracy: 0.0860
Epoch 3/500
2887/2891 [============================>.] - ETA: 0s - loss: 6.4663 - accuracy: 0.0796
Epoch 00003: accuracy improved from 0.07739 to 0.07960, saving model to nlp_anthro_v2.h5
2891/2891 [==============================] - 47s 16ms/step - loss: 6.4665 - accuracy: 0.0796 - val_loss: 7.2691 - val_accuracy: 0.0994
Epoch 4/500
2890/2891 [============================>.] - ETA: 0s - loss:

In [ ]:
#Rede robusta:

model_2 = Sequential()
model_2.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model_2.add(LSTM(250,return_sequences=True))
model_2.add(LSTM(250))
model_2.add(Dense(250,activation='relu'))
model_2.add(Dense(vocabulary_size, activation='softmax'))
print(model_2.summary())
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 3, 3)              31041     
                                                                 
 lstm_6 (LSTM)               (None, 3, 250)            254000    
                                                                 
 lstm_7 (LSTM)               (None, 250)               501000    
                                                                 
 dense_6 (Dense)             (None, 250)               62750     
                                                                 
 dense_7 (Dense)             (None, 10347)             2597097   
                                                                 
Total params: 3,445,888
Trainable params: 3,445,888
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Treinando:
early = callbacks.EarlyStopping(monitor = 'accuracy',patience= 5)
check = callbacks.ModelCheckpoint(filepath = 'nlp_anthro_v3.h5', monitor = 'accuracy', save_best_only = True, verbose = 1, mode='auto')            



model_2.fit(train_inputs,train_targets,epochs=500,validation_split=0.3,verbose=1, callbacks= [early,check])

Epoch 1/500
2891/2891 [==============================] - ETA: 0s - loss: 6.7729 - accuracy: 0.0794
Epoch 00001: accuracy improved from -inf to 0.07945, saving model to nlp_anthro_v3.h5
2891/2891 [==============================] - 223s 76ms/step - loss: 6.7729 - accuracy: 0.0794 - val_loss: 6.8652 - val_accuracy: 0.1047
Epoch 2/500
2891/2891 [==============================] - ETA: 0s - loss: 6.3352 - accuracy: 0.0979
Epoch 00002: accuracy improved from 0.07945 to 0.09787, saving model to nlp_anthro_v3.h5
2891/2891 [==============================] - 214s 74ms/step - loss: 6.3352 - accuracy: 0.0979 - val_loss: 6.7494 - val_accuracy: 0.1152
Epoch 3/500
2891/2891 [==============================] - ETA: 0s - loss: 6.0167 - accuracy: 0.1154
Epoch 00003: accuracy improved from 0.09787 to 0.11535, saving model to nlp_anthro_v3.h5
2891/2891 [==============================] - 216s 75ms/step - loss: 6.0167 - accuracy: 0.1154 - val_loss: 6.8408 - val_accuracy: 0.1245
Epoch 4/500
2891/2891 [========

In [ ]:
#pequena função para fazer predições utilizando inputs:

def word_pred(model_name):
  input_text = input().strip().lower() #recebe o input, separa e põe minúsculo:
  encoded_text = tokenizer.texts_to_sequences([input_text])[0] #passa pelo tokenizer para criar a sequência
  pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre') #Faz o encode final:
  print(encoded_text, pad_encoded)
  for i in (model_name.predict(pad_encoded)[0]).argsort()[-3:][::-1]: #Faz o predict das 3 palavras associadas:
    pred_word = tokenizer.index_word[i]
    print("Next word suggestion:",pred_word)

In [ ]:
model_v1 = models.load_model('nlp_anthro_v1.h5') #Rede padrão:
model_v2 = models.load_model('nlp_anthro_v2.h5') #Rede compacta:
model_v3 = models.load_model('nlp_anthro_v3.h5') #Rede robusta:

Abaixo faremos algumas predições com palavras chaves. Em cada célula faremos 3 predições (uma em cada rede) utilizando a mesma palavra para compararmos resultados.

In [ ]:
#africans
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

Africans
[474] [[  0   0 474]]
Next word suggestion: is
Next word suggestion: are
Next word suggestion: and
Africans
[474] [[  0   0 474]]
Next word suggestion: and
Next word suggestion: the
Next word suggestion: in
Africans
[474] [[  0   0 474]]
Next word suggestion: remained
Next word suggestion: who
Next word suggestion: their


In [ ]:
#black
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

black
[154] [[  0   0 154]]
Next word suggestion: youths
Next word suggestion: heads
Next word suggestion: clothes
black
[154] [[  0   0 154]]
Next word suggestion: great
Next word suggestion: few
Next word suggestion: lion
black
[154] [[  0   0 154]]
Next word suggestion: pigmy
Next word suggestion: children
Next word suggestion: leaves


In [ ]:
#she
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

she
[86] [[ 0  0 86]]
Next word suggestion: escaped
Next word suggestion: was
Next word suggestion: did
she
[86] [[ 0  0 86]]
Next word suggestion: and
Next word suggestion: the
Next word suggestion: is
she
[86] [[ 0  0 86]]
Next word suggestion: evidently
Next word suggestion: is
Next word suggestion: gutenberg


In [ ]:
#he
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

he
[14] [[ 0  0 14]]
Next word suggestion: lays
Next word suggestion: speaks
Next word suggestion: owes
he
[14] [[ 0  0 14]]
Next word suggestion: and
Next word suggestion: the
Next word suggestion: is
he
[14] [[ 0  0 14]]
Next word suggestion: need
Next word suggestion: gave
Next word suggestion: fell


In [ ]:
#sex
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

sex
[247] [[  0   0 247]]
Next word suggestion: promiscuity
Next word suggestion: fun
Next word suggestion: air
sex
[247] [[  0   0 247]]
Next word suggestion: scylla
Next word suggestion: fifth
Next word suggestion: p
sex
[247] [[  0   0 247]]
Next word suggestion: is
Next word suggestion: need
Next word suggestion: distinctly


In [ ]:
#Family
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)


family
[689] [[  0   0 689]]
Next word suggestion: is
Next word suggestion: as
Next word suggestion: among
family
[689] [[  0   0 689]]
Next word suggestion: the
Next word suggestion: and
Next word suggestion: in
family
[689] [[  0   0 689]]
Next word suggestion: an
Next word suggestion: it
Next word suggestion: which


In [ ]:
#primitive
word_pred(model_v1)
word_pred(model_v2)
word_pred(model_v3)

primitive
[160] [[  0   0 160]]
Next word suggestion: youths
Next word suggestion: g
Next word suggestion: clothes
primitive
[160] [[  0   0 160]]
Next word suggestion: great
Next word suggestion: girls
Next word suggestion: children
primitive
[160] [[  0   0 160]]
Next word suggestion: food
Next word suggestion: people
Next word suggestion: children


Considerações finais.

As palavras acima são palavras chaves para os livros escolhidos. Nelas podemos ver a diferença dos 3 treinos. A v1 padrão prediz palavras com algum sentido, bem diferente da segunda versão mais compacta, que tem a maior parte das predições com palavras conectivas muitas vezes sem produzir qualquer sentido. 
<br>
Já a versão 3 sofreu um overfitting em seu treino, sua accuracy não parou de subir até alcançar patamares maiores que 0.60, ainda sim, seu resultados não são descartáveis como aconteceria em casos onde a avaliação das predições é lógica ou numérica. Vemos algum sentido nas palavras preditas e isso não pode ser descartado.
<br><br>
A análise do modelo padrão (v1) sobre a palavra sex é bem interessante. É sabido do choque cultural que os europeus levaram ao longo e após as grandes navegações com os povos de outros continentes. A palavra sex ter como predições as palavras "promiscuity","fun" e "air". Diz muito sobre os hábitos sexuais dos nativos de outras terras e de como isso chocava o europeu no papel de observador e ao mesmo tempo opressor, já que essas informações foram utilizadas (e ainda são) para a dominação destes povos, seja através do espanto diante da diferença cultural, seja utilizando argumentos que alegam a não-civilização. É interessante perceber que esta análise não foi feita sobre a predição em si, mas demonstra que a predição da v1 está contundente com a realidade dos fatos já conhecidos.
<br><br>
É interessante perceber ainda com a v1 que a primeira predição para a palavra "she" é escaped. Um termo bem sugestivo, relacionado a violência, e ao perigo. Enquanto a palavra "he" retorna "lays". Enquanto ela foge, ele se deita. Ainda que não valha a pena fazer mais análises, a versão 1 mostra-se bem calibrada em termos de sentido.
<br><br>
As versões 2 e 3, pecam pela falta e pelo excesso respectivamente, a mais compacta ganha em velocidade de treinamento e perde em análise, já que suas predições serão de conectivos em maior parte. <br> Já a versão 3 mais robusta, tendeu para um overfitting e prediz algumas palavras vazias, que não produzem muito sentido mas ainda sim cabíveis. A palavra primitive mostra que todas as redes tem alguma efetividade, já que, apesar dela não ser a mais comum ela é de caráter central para a semântica da maioria dos textos, e traz predições condizentes com o esperado em todas as redes.
<br><br><br>

NLPs como as feitas aqui são muito úteis em aplicativos de mensagens instantâneas, análises automatizadas de textos, resumos e papers em geral. Neste nosso caso serviu para uma pequena análise semântica, mas com este mesmo exemplo poderíamos melhorar o vocabulário na rede para formar frases e até mesmo aprender a escrever como um escritor em específico faria se alimentássemos a rede apenas com textos dele, como muitas outras aplicações.

<br><br>
Obrigado pela atenção! ;)